In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

path = 'C:/Users/queenSSH/Desktop/학교/CUAI/cuai하계/'

naver = pd.read_csv(path + '네이버_추천시스템.csv' , encoding='cp949')
daum = pd.read_csv(path + 'daum_final.csv', encoding = 'cp949')

naver = naver[['title', 'story']]
daum = daum[['title','story']]

#web_combined = pd.concat([naver, daum], axis=0, ignore_index=True)
web_combined = daum
#web_combined = naver

In [2]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np

class SentenceTokenizer(object):
    def __init__(self) -> None:
        self.kkma = Kkma()
        self.twitter = Twitter()
#         self.stopwords = ['중인', '만큼', '마찬가지', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가']
        self.stopwords = ['을', '를', '에', '의', '가','우리','저희','그녀']

#     def excel2sentences(self, file):
#         story = pd.read_csv(file,  encoding='cp949')
#         story = story['story']
#         #story.parse()
#         #sentences = self.kkma.sentences(story.text)
#         sentences = self.kkma.sentences(story)

#         for i in range(0, len(sentences)):
#             if len(sentences[i]) <= 10:
#                 sentences[i - 1] += (' ' + sentences[i])
#                 sentences[i] = ''

#         return sentences

    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for i in range(0, len(sentences)):
            if len(sentences[i]) <= 10:
                sentences[i - 1] += (' ' + sentences[i])
                sentences[i] = ''

        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence)) if noun not in self.stopwords and len(noun) > 1]))

        return nouns



In [3]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]

        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            
            if link_sum != 0:
                A[:, id] /= link_sum
            
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

    def keywords(self, word_num=5):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords


In [4]:
print(web_combined['story'][0])

너무 다른 네 명의 절친이 뜬금없이 카페 동업하면서 벌어지는 좌충우돌 청춘드라마


In [5]:
# key_wordlist = []
# import warnings
# warnings.filterwarnings('ignore')

# for i in range(0, len(web_combined['story'])):
#     try :
        
#         text = web_combined['story'][i]
#         textrank = TextRank(text)
#         print('keywords :', textrank.keywords())
#         key_wordlist.append(textrank.keywords())
        
#     except:
#         key_wordlist.append("none")
    

## Naver

In [6]:
#naver1 = pd.read_csv(path + '네이버_추천시스템.csv' , encoding='cp949')

In [7]:
#naver1['keywords'] = key_wordlist

In [8]:
#naver1.to_csv('네이버_웹툰_최최종.csv')

## Daum

In [9]:
key_wordlist2 = []
import warnings
warnings.filterwarnings('ignore')

for i in range(0, len(web_combined['story'])):
    try :
        
        text = web_combined['story'][i]
        textrank = TextRank(text)
        print('keywords :', textrank.keywords())
        key_wordlist2.append(textrank.keywords())
        
    except:
        key_wordlist2.append("none")

keywords : ['좌충우돌', '카페', '다른', '동업', '드라마']
keywords : ['덕질', '마음', '아이', '위해']
keywords : ['선수', '방송', '요한', '졸업', '지망']
keywords : ['악마', '사회', '위협', '적응', '존재']
keywords : ['다시', '사내', '수저', '천하', '제일']
keywords : ['무엇', '정체', '자꾸만', '의사', '주변']
keywords : ['노예', '명월', '전국시대', '기생', '무림']
keywords : ['세상', '작가', '조경', '최선']
keywords : ['고전', '단영', '복수', '성장', '오누이']
keywords : ['식당', '백로', '신들', '요리', '음식']
keywords : ['구역', '남자', '사랑', '여자', '수거']
keywords : ['도전', '리스', '시험', '주인공', '전투']
keywords : ['라이프', '조선시대', '집사', '나무꾼', '동물']
keywords : ['소녀', '시스트', '엑소', '이야기']
keywords : ['로맨스', '발레리', '철벽', '환생']
keywords : ['댄스', '거머', '우승', '위드', '파트너']
keywords : ['재능', '합작', '그린', '미술', '스타']
keywords : ['감염', '바이러스', '소가', '존재', '수용']
keywords : ['세상', '신인', '어차피', '제정']
keywords : ['귀신', '숙경', '이야기', '학교', '존재']
keywords : ['윤정', '이야기', '학교']
keywords : ['녀석', '시작', '아빠', '엄마']
keywords : ['공개', '낚는', '라면', '청산', '생존']
keywords : ['도움', '귀신', '소년', '의문', '학교']
keywords : ['데트', 

keywords : ['개장', '동물', '동물원']
keywords : ['가슴', '로맨스', '시대극', '진실', '애증']
keywords : ['눈빛', '아이']
keywords : ['유령', '천재', '귀신', '라며', '미녀']
keywords : ['뷰티', '크리', '관심', '그림', '로맨스']
keywords : ['기회', '남녀', '눈앞', '모든', '때마침']
keywords : ['어린이', '예술가', '욕망', '하나']
keywords : ['오늘', '혹시']
keywords : ['시대', '몸부림', '시작', '욕망', '짐승']
keywords : ['끼리', '밌으', '세상', '끼리끼리', '츠기']
keywords : ['세계', '승은', '후궁']
keywords : ['남녀', '사고', '안개', '탈출']
keywords : ['가장', '다른', '자신', '존재', '장소']
keywords : ['궁합', '다른', '성격', '연상', '연하남']
keywords : ['남자', '십년', '외출']
keywords : ['공인', '데뷔', '사건', '일대', '자타']
keywords : ['액션', '코믹', '간의', '기억', '드일행']
keywords : ['껍데기', '도하', '성형', '의사', '외과']
keywords : ['매몰', '발신', '전화', '표시', '제한']
keywords : ['등골', '기어', '벌레', '오싹', '찰나']
keywords : ['남자', '라이프', '서울', '여자', '시골']
keywords : ['대판', '대하', '심층', '논리', '만화']
keywords : ['규칙', '노인', '매우', '아이']
keywords : ['소녀', '이야기', '태생', '행복']
keywords : ['악연', '유기한', '남자', '사채', '상처']
keywords : ['다시', '스무', '오기']


keywords : ['도시', '무조건', '아웃', '화이트', '전체']
keywords : ['본적', '구조', '산악', '이드', '이제껏']
keywords : ['다른', '동물', '메트로놈', '원진', '반려']
keywords : ['화성']
keywords : ['개인', '나태', '다큐', '서울대']
keywords : ['세기', '홈즈', '가상', '각종', '국가']
keywords : ['은혜', '형제', '놀부', '부와', '사이']
keywords : ['가지', '단편', '버스', '사람']
keywords : ['동구', '네온', '부부', '자동차', '캐러멜']
keywords : ['드비', '이야기', '일상', '작가']
keywords : ['명예', '수만', '지위']
keywords : ['상실', '오감', '강의', '기억', '누구']
keywords : ['교향악단', '시립', '음악', '이야기']
keywords : ['다운', '드라마', '미스', '변호사', '법정']
keywords : ['바로', '순금']
keywords : ['사무소', '사건', '사설', '이야기', '탐정']
keywords : ['기록', '기네스', '기상', '외한', '이야기']
keywords : ['공모전', '그날', '다음', '상작', '최우수']
keywords : ['교실', '여왕', '정말', '지배']
keywords : ['고양이', '당신', '모든']
keywords : ['각자', '복수', '시작', '싸움']
keywords : ['남자', '마녀', '사랑', '이야기', '여자']
keywords : ['누구', '로맨스', '사랑', '판타지']
keywords : ['나름', '녀석', '사정', '여교사']
keywords : ['배팅', '당첨', '복권', '인생', '자의']
keywords : ['남자', '모든', '선택', '여자']
key

keywords : ['고통', '기생', '김삿갓', '영월', '조선']
keywords : ['달리', '준수', '드라마', '성장', '푸른']
keywords : ['드립', '끝판', '순도', '유희', '키몽']
keywords : ['묵직', '사생활', '웹툰작가']
keywords : ['계세', '당신', '대해', '얼마나', '사람']
keywords : ['동물', '각양각색', '드라마', '버스', '야생']
keywords : ['기관', '대립', '욕망', '권력', '사회']
keywords : ['액션', '의학', '국제', '대한민국', '무도']
keywords : ['세상', '악마', '엘리', '일대기', '영웅']
keywords : ['낭만', '귀신', '대책', '처녀', '취준생']
keywords : ['스릴러', '위기', '이야기', '전쟁', '첩보']
keywords : ['랜드', '스코', '로드', '버디', '트립']
keywords : ['스토리', '스트랄', '우먼', '파워', '친구']
keywords : ['기간', '발표', '시험', '정답', '지급']
keywords : ['그림', '대항', '스님', '일제', '이야기']
keywords : ['공포', '당신', '순간', '일상']
keywords : ['나로', '게임', '브랜드', '온라인', '진혼곡']
keywords : ['갤럭시', '기상', '라디오', '방송', '외한']
keywords : ['난기', '경찰', '대처', '재난', '지구인']
keywords : ['분식집', '소동', '연애']
keywords : ['사랑', '기적', '위선', '위해']
keywords : ['과연', '누구', '범인', '진짜', '용의자']
keywords : ['병사', '설봉', '고려', '마을', '무인도']
keywords : ['다음웹툰', '우주', '져스', '콜라보', '지구'

In [10]:
daum1 = pd.read_csv(path + 'daum_final.csv', encoding = 'cp949')

daum1['keywords'] = key_wordlist2

In [11]:
daum1.to_csv('다음_웹툰_최최종.csv')

In [12]:
# import warnings
# warnings.filterwarnings('ignore')

# for i in range(0, len(web_combined['story'])):
#     text = web_combined['story'][i]

#     textrank = TextRank(text)

#     print('keywords :', textrank.keywords())